# Programação Linear Inteira

Problemas de otimização linear que exigem que algumas das variáveis sejam números inteiros são chamados de Programas Mistos Inteiros (MIPs). Essas variáveis podem surgir de duas maneiras:

- **Variáveis inteiras** que representam o número de itens, como carros ou aparelhos de televisão, onde o problema é decidir quantos de cada item fabricar para maximizar o lucro. Normalmente, esses problemas podem ser configurados como problemas de otimização linear padrão, com o requisito adicional de que as variáveis devem ser números inteiros. A próxima seção mostra um exemplo desse tipo de problema.

- **Variáveis booleanas** que representam decisões com valores 0 ou 1. Como exemplo, considere um problema que envolve a atribuição de trabalhadores a tarefas. Para configurar esse tipo de problema, você pode definir variáveis booleanas $x_{i,j}$ iguais a 1 se o trabalhador i estiver designado à tarefa j e 0, caso contrário.

O Google OR-Tools fornece duas opções para resolver MIPs:

- MPSolver: um conjunto de solucionadores MIP de terceiros, que usam técnicas de Branch-and-Bound. Este consiste em uma enumeração sistemática de candidatos a solução, através da qual grandes subconjuntos de candidatos infrutíferos são descartados utilizando os limites superior e inferior da quantia otimizada.

- CP-SAT: um solucionador de Programação de Restrições (Constraint Programming) que usa métodos SAT (satisfabilidade booleana). O problema de satisfatibilidade booleana é o problema de determinar se existe uma determinada valoração para as variáveis de uma determinada fórmula booleana tal que esta valoração satisfaça esta fórmula em questão. Por exemplo, tomando $x_{1},x_{2},x_{3},x_{4}$ como as variáveis booleanas e a expressão $(x_{1}\lor \neg x_{3}\lor x_{4})\land (\neg x_{2}\lor x_{3}\lor \neg x_{4})$ caso exista uma atribuição de valores 0 ou 1 para as variáveis da fórmula que torne a fórmula avaliada VERDADEIRA (1), esta fórmula é considera satisfatível, em contrapartida se nenhuma atribuição levou a uma avaliação da fórmula como verdadeira, ela é considerada insatisfatível.

## MIP Solver

As seções a seguir apresentam um exemplo de um MIP e mostram como resolvê-lo. Aqui está o problema:

### Maximize $x + 10y$, sujeito às seguintes restrições
$
\begin{align}
x + 7 y	\le	17.5\\
x	\le	3.5\\
x	\ge	0\\
y	\ge	0\\
x, y \in \mathbb{N} 
\end{align}
$

Como as restrições são lineares, esse é apenas um problema de otimização linear, no qual as soluções precisam ser inteiras. 

O gráfico abaixo mostra os pontos inteiros na região viável para o problema.

<img style="float: center;" src="https://developers.google.com/optimization/images/mip/feasible_region.png">

Observe que esse problema é muito semelhante ao problema resolvido descrito em [Introdução a Otimização Linear](aula-01-intro-po.ipynb), mas neste caso exigimos que as soluções sejam inteiras.

## As seções a seguir explicam como resolver o problema passo a passo.
0. Incluir as bibliotecas

In [1]:
#include "setup.h"
#include "ortools/linear_solver/linear_solver.h"
#include "ortools/linear_solver/linear_solver.pb.h"
#include <iostream>
using namespace operations_research;

1. Escolha o solver.
> O código a seguir declara o solucionador. O MPsolver é um invólucro para vários solucionadores diferentes, incluindo o CBC. Escolha CBC_MIXED_INTEGER_PROGRAMMING para informar ao solucionador para usar MIP.

In [2]:
MPSolver solver("simple_mip_program", MPSolver::CBC_MIXED_INTEGER_PROGRAMMING);

2. Crie as variáveis.
- use o método MakeIntVar para criar variáveis x e y que assumem valores inteiros não negativos.
- defina os limites inferiores e superiores das variáveis x e y no intervalo de 0 a infinito. 


In [3]:
const double infinity = solver.infinity();

MPVariable* const x = solver.MakeIntVar(0.0, infinity, "x");
MPVariable* const y = solver.MakeIntVar(0.0, infinity, "y");

std::cout << "Number of variables = " << solver.NumVariables();

Number of variables = 2

3. Defina as restrições.
> defina as restrições das variáveis. Atribua a cada restrição um nome exclusivo (como restrição 0) e defina os coeficientes para a restrição.

In [4]:
// x + 7 * y <= 17.5.
MPConstraint* const c0 = solver.MakeRowConstraint(-infinity, 17.5, "c0");
c0->SetCoefficient(x, 1);
c0->SetCoefficient(y, 7);

// x <= 3.5.
MPConstraint* const c1 = solver.MakeRowConstraint(-infinity, 3.5, "c1");
c1->SetCoefficient(x, 1);
c1->SetCoefficient(y, 0);

std::cout << "Number of constraints = " << solver.NumConstraints();

Number of constraints = 2

4. Defina a função objetivo.
> O código a seguir define a função objetivo, $Z = x + 10y$, e especifica que este é um problema de maximização.

In [5]:
// Maximize x + 10 * y.
MPObjective* const objective = solver.MutableObjective();
objective->SetCoefficient(x, 1);
objective->SetCoefficient(y, 10);
objective->SetMaximization();

5. Execute o solver e exiba os resultados.
> O código a seguir chama o solver e exibe a solução.

In [6]:
const MPSolver::ResultStatus result_status = solver.Solve();
// Check that the problem has an optimal solution.
if (result_status != MPSolver::OPTIMAL) {
     std::cout << "The problem does not have an optimal solution!";
}

6. Exibir a solução
> O código a seguir exibe a solução com valor ótimo da função objetivo 23, o que ocorre no ponto x = 3, y = 2.

In [7]:
std::cout << "Solution:" << std::endl;
std::cout << "Objective value = " << objective->Value() << std::endl;
std::cout << "x = " << x->solution_value() << std::endl;
std::cout << "y = " << y->solution_value() << std::endl;

std::cout << "\nAdvanced usage:" << std::endl;
std::cout << "Problem solved in " << solver.wall_time() << " milliseconds" << std::endl;
std::cout << "Problem solved in " << solver.iterations() << " iterations" << std::endl;
std::cout << "Problem solved in " << solver.nodes() << " branch-and-bound nodes" << std::endl;

Solution:
Objective value = 23
x = 3
y = 2

Advanced usage:
Problem solved in 124686 milliseconds
Problem solved in 1 iterations
Problem solved in 0 branch-and-bound nodes


### Comparando programação linear e inteira

Vamos comparar a solução com o problema de otimização com número inteiros, mostrado acima, com a solução  do problema de otimização linear correspondente, no qual as restrições de número inteiro são removidas.

Você pode pensar a princípio que a solução para o problema inteiro seria o ponto de coordenadas inteiras na região viável mais próxima da solução com números reais - ou seja, o ponto x = 0, y = 2. Mas, como você verá a seguir, este não é o caso.

Você pode modificar facilmente o programa na seção anterior para resolver o problema linear com números reais, fazendo as seguintes alterações:

1. Substitua o solucionador MIP com o solucionador LP
2. Substitua as variáveis inteiras discretas por variáveis reais contínuas

In [ ]:
//Implemente aqui as alterações

Após fazer essas alterações e executar o programa novamente, você obtém a seguinte saída:

A solução para o problema linear ocorre no ponto x = 0, y = 2,5, onde a função objetivo é igual a 25. 

Aqui está um gráfico mostrando as soluções para os problemas linear e inteiro.
<img style="float: center;" src="https://developers.google.com/optimization/images/mip/feasible_region_sol.png">

Observe que a solução inteira não está próxima da solução com números reais, comparada com a maioria dos outros pontos inteiros na região viável. Em geral, as soluções para um problema de otimização linear com números reais e os problemas de otimização com números inteiros correspondentes podem estar distantes. Por esse motivo, os dois tipos de problemas exigem métodos diferentes para sua solução.

## CP-SAT

### Maximize 2x + 2y + 3z sujeito às seguintes restrições:
$
\begin{align}
x + \frac{7}{2} y + \frac{3}{2} z	\le	25\\
3x - 5y + 7z	\le	45\\
5x + 2y - 6z	\le	37\\
x, y, z \ge	0\\
x, y, z \in \mathbb{N} 
\end{align}
$

Para aumentar a velocidade computacional, o solucionador CP-SAT trabalha com números inteiros. Isso significa que todas as restrições e o objetivo devem ter coeficientes inteiros. 

No exemplo acima, a primeira restrição não atende a essa condição. 

Para resolver o problema, você deve primeiro transformar a restrição multiplicando-a por um número inteiro suficientemente grande para converter todos os coeficientes em números inteiros. 

Isso é mostrado na seção Restrições abaixo.

In [8]:
#include "setup.h"
#include "ortools/sat/cp_model.h"
#include <iostream>
using namespace operations_research;

In [9]:
{
    sat::CpModelBuilder cp_model;

    const Domain domain(0, 999);
    const sat::IntVar x = cp_model.NewIntVar(domain).WithName("x");
    const sat::IntVar y = cp_model.NewIntVar(domain).WithName("y");
    const sat::IntVar z = cp_model.NewIntVar(domain).WithName("z");

    cp_model.AddLessOrEqual(sat::LinearExpr::ScalProd({x, y, z}, {2, 7, 3}),50); //2*x + 7*y + 3*z <= 50
    cp_model.AddLessOrEqual(sat::LinearExpr::ScalProd({x, y, z}, {3, -5, 7}),45); //3*x - 5*y + 7*z <= 45
    cp_model.AddLessOrEqual(sat::LinearExpr::ScalProd({x, y, z}, {5, 2, -6}),37); //5*x + 2*y - 6*z <= 37

    cp_model.Maximize(sat::LinearExpr::ScalProd({x, y, z}, {2, 2, 3})); //2*x + 2*y + 3*z

    const sat::CpSolverResponse response = Solve(cp_model); 

    if (response.status() == sat::CpSolverStatus::OPTIMAL) { 
        std::cout << "FO:" << response.objective_value() << std::endl;
        std::cout << "x: " << SolutionIntegerValue(response, x) << std::endl;
        std::cout << "y: " << SolutionIntegerValue(response, y) << std::endl; 
        std::cout << "z: " << SolutionIntegerValue(response, z) << std::endl; 
    }
}

FO:35
x: 7
y: 3
z: 5
